In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
import datetime as dt

from glob import iglob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Get MMI data
df_mmi=df[df['Type']=='meanMotionIntensity']

In [ ]:
df_mmi['Start Time']=df_mmi['Start Time'].apply(pd.to_datetime)

In [ ]:
df_mmi['End Time']=df_mmi['End Time'].apply(pd.to_datetime)

In [ ]:
df_mmi['Start_Time_Offset']=df_mmi['Start Time']+ pd.to_timedelta(df_mmi['Time Zone Offset'], unit='s')
df_mmi['End_Time_Offset']=df_mmi['End Time']+ pd.to_timedelta(df_mmi['Time Zone Offset'], unit='s')

In [ ]:
#get steps data
df_steps=df[df['Type']=='steps']
df_steps['User UUID'].nunique()

In [ ]:
df_steps['Start Time']=df_steps['Start Time'].apply(pd.to_datetime)

In [ ]:
df_steps['End Time']=df_steps['End Time'].apply(pd.to_datetime)

In [ ]:
df_steps['Start_Time_Offset']=df_steps['Start Time']+ pd.to_timedelta(df_steps['Time Zone Offset'], unit='s')
df_steps['End_Time_Offset']=df_steps['End Time']+ pd.to_timedelta(df_steps['Time Zone Offset'], unit='s')

In [ ]:
df_merge_steps_mi = pd.merge(df_steps, df_mmi, left_on=['User UUID','Start_Time_Offset','End_Time_Offset'],right_on=['User UUID','Start_Time_Offset','End_Time_Offset'])
df_merge_steps_mi
df_merge_steps_mi['User UUID'].nunique()

In [ ]:
df_merge_steps_mi

In [ ]:

df_merge_dup = df_merge_steps_mi.drop_duplicates(subset=['User UUID', 'Start_Time_Offset','End_Time_Offset','Value_x'], keep='first')
df_merge_dup

In [ ]:
df_merge_dup['End Time']=df_merge_dup['End Time_x'].apply(pd.to_datetime)

In [ ]:
df_merge_dup['Start_Time_Date_Offset_x'] = df_merge_dup['Start_Time_Offset'].dt.date
df_merge_dup['End_Time_Date_Offset_x'] = df_merge_dup['End_Time_Offset'].dt.date

In [ ]:
df_merge_dup['month_year'] = df_merge_dup['Start_Time_Offset'].dt.to_period('M')

df_merge_dup['Total_Steps_Per_Day_pre_missing'] = df_merge_dup.groupby(['Start_Time_Date_Offset_x', 'User UUID'])['Value_x'].transform('sum')



df_merge_dup['Mon_Year'] = df_merge_dup['Start_Time_Offset'].dt.strftime('%b-%Y')

df_merge_dup['Month_Year_Name']=df_merge_dup['Mon_Year']

df_merge_dup.loc[(df_merge_dup['Start_Time_Date_Offset_x'] <dt.date(2020,3,15)) & (df_merge_dup['Start_Time_Date_Offset_x'] >=dt.date(2020,3,1)), 'Month_Year_Name'] = '1-14 Mar-2020'
df_merge_dup.loc[(df_merge_dup['Start_Time_Date_Offset_x'] <dt.date(2020,4,1)) & (df_merge_dup['Start_Time_Date_Offset_x'] >dt.date(2020,3,14)), 'Month_Year_Name'] = '15-31 Mar-2020'
df_merge_dup.loc[(df_merge_dup['Start_Time_Date_Offset_x'] >=dt.date(2020,4,1)), 'Month_Year_Name'] = df_merge_dup.Month_Year_Name


df_merge_dup['Average_Monthly_Steps_Lockdown_pre_missing']=df_merge_dup.groupby([ 'User UUID','Month_Year_Name'])['Total_Steps_Per_Day_pre_missing'].transform('mean')


In [ ]:
df_merge_dup['Average_Monthly_Steps_Lockdown_pre_missing'].mean()


In [ ]:
df_merge_dup['Average_Monthly_Steps_Lockdown_pre_missing'].std()

In [ ]:
df_merge_dup['Average_Monthly_Steps_Lockdown_pre_missing'].min()

In [ ]:
df_merge_dup['Average_Monthly_Steps_Lockdown_pre_missing'].max()

In [ ]:
df_merge_dup['User UUID'].nunique()

In [ ]:
df_zero=df_merge_dup[df_merge_dup['Value_y']<1]

In [ ]:
df_non_zero=df_merge_dup[df_merge_dup['Value_y']>=1]
df_non_zero
df_non_zero['User UUID'].nunique()

In [ ]:
#Calculate missingness
#df_steps_missing=df_merge_steps_mi_dup_non_zero
df_steps_missing=df_non_zero


df_steps_missing['Num_of_days']=0
df_steps_missing['total_percent_missing']=0
df_steps_missing['percent_missing_per_day']=0
df_steps_missing['Epochs_per_day']=0


df_steps_missing['Epochs_per_day_count']=df_steps_missing.groupby([ 'User UUID','Start_Time_Date_Offset_x'])['Start_Time_Offset'].transform('count')
df_steps_missing['Epochs_per_day']=df_steps_missing.groupby([ 'User UUID','Start_Time_Date_Offset_x'])['Start_Time_Offset'].transform('nunique')
df_steps_missing['Epochs_per_day']=pd.to_numeric(df_steps_missing['Epochs_per_day'])
df_steps_missing['percent_missing_per_day']  = df_steps_missing['Epochs_per_day']/96
df_steps_missing['Num_of_days']=df_steps_missing.groupby([ 'User UUID'])['Start_Time_Date_Offset_x'].transform('nunique')



In [ ]:
df_steps_missing['Epochs_per_day'].max()

In [ ]:
df_steps_missing['Epochs_per_day'].min()

In [ ]:
df_steps_missing['Valid_Day']='no'
df_steps_missing.loc[(df_steps_missing['Epochs_per_day'] >40), 'Valid_Day'] = 'yes'


In [ ]:
#restrict time
df_all_periods=df_steps_missing[(df_steps_missing['Start_Time_Date_Offset_x']>=dt.date(2019,3,1)) & (df_steps_missing['Start_Time_Date_Offset_x'] <=dt.date(2021,6,30))]

print(df_all_periods['Start_Time_Date_Offset_x'].min())

print(df_all_periods['Start_Time_Date_Offset_x'].max())



In [ ]:
df_all_periods.loc[(df_all_periods['Start_Time_Date_Offset_x'] <dt.date(2020,3,15)) & (df_all_periods['Start_Time_Date_Offset_x'] >=dt.date(2019,3,1)), 'closure_status'] = 'Pre Closure'

df_all_periods.loc[(df_all_periods['Start_Time_Date_Offset_x'] <=dt.date(2021,3,31)) & (df_all_periods['Start_Time_Date_Offset_x'] >=dt.date(2020,3,15)), 'closure_status'] = 'During Closure'

df_all_periods.loc[(df_all_periods['Start_Time_Date_Offset_x'] >dt.date(2021,3,31)), 'closure_status'] = 'Post Closure'

print(df_all_periods.groupby('closure_status')['Start_Time_Date_Offset_x'].min())


In [ ]:
print(df_all_periods.groupby('closure_status')['Start_Time_Date_Offset_x'].max())

In [ ]:
df_valid_days=df_all_periods[df_all_periods['Valid_Day']=='yes']


df_valid_days['Num_of_days_valid_epochs']=df_valid_days.groupby(['User UUID'])['Start_Time_Date_Offset_x'].transform('nunique')


df_valid_days_daycount=df_valid_days[df_valid_days['Num_of_days_valid_epochs']>60]

In [ ]:
df_valid_days_daycount['Epochs_per_day'].min()

In [ ]:
df_valid_days_daycount['Epochs_per_day'].max()

In [ ]:
epoch_count=df_all_periods
epoch_count['Num_of_days_valid_epochs']=epoch_count.groupby(['User UUID'])['Start_Time_Date_Offset_x'].transform('nunique')

epoch_count = df_all_periods.drop_duplicates(subset=['User UUID', 'Num_of_days_valid_epochs'], keep='first')
epoch_count

epoch_count_only=epoch_count[['Num_of_days_valid_epochs']]
epoch_count_only




In [ ]:

plt.figure(figsize=(60,20))
sns.set(font_scale =5)
default_x_ticks = range(len(epoch_count_only))
fig, ax = plt.subplots(figsize=(60,20))
ax.hist(epoch_count_only['Num_of_days_valid_epochs'],bins=61,cumulative=-1, density=False)
ax.set_facecolor('white')



In [ ]:
epoch_count_only[epoch_count_only['Num_of_days_valid_epochs']>=61].nunique()

In [ ]:
epoch_count_only[epoch_count_only['Num_of_days_valid_epochs']>=40].nunique()

In [ ]:
epoch_count_only[epoch_count_only['Num_of_days_valid_epochs']>=70].nunique()

In [ ]:
epoch_count_only[epoch_count_only['Num_of_days_valid_epochs']>=80].nunique()

In [ ]:
epoch_count_only[epoch_count_only['Num_of_days_valid_epochs']>=90].nunique()

In [ ]:
#Heatmap

df_epoch_dup=df_all_periods.drop_duplicates(subset=['User UUID', 'Start_Time_Date_Offset_x'], keep='last',inplace=False)
df2=df_epoch_dup.groupby(['User UUID','Epochs_per_day'])['Start_Time_Date_Offset_x'].count()

df3=df2.reset_index()

df3["Epochs_per_day"] = df3["Epochs_per_day"].replace(np.nan, 0)

df3["Start_Time_Date_Offset_x"] = df3["Start_Time_Date_Offset_x"].replace(np.nan, 0)


df_pivot=df3.pivot_table(values='User UUID', index=['Epochs_per_day'],
                    columns=['Start_Time_Date_Offset_x'], aggfunc=pd.Series.nunique)
df_pivot.replace(np.nan, 0,inplace=True)

sns.set(rc={"figure.figsize":(20, 7)})

sns.set_context("paper", rc={"font.size":90,"axes.titlesize":50,"axes.labelsize":20})
sns.set(rc={'axes.facecolor':'snow', 'figure.facecolor':'white'})
ax=sns.heatmap(df_pivot, cmap="rocket_r", mask=(df_pivot==0))
ax.invert_yaxis()

In [ ]:
df_valid_days_daycount['User UUID'].nunique()

In [ ]:
df_valid_days_daycount['Total_Steps_Per_Day_post_missing'] = df_valid_days_daycount.groupby(['Start_Time_Date_Offset_x', 'User UUID'])['Value_x'].transform('sum')


df_valid_days_daycount['Average_Monthly_Steps_Lockdown_post_missing']=df_valid_days_daycount.groupby([ 'User UUID','Month_Year_Name'])['Total_Steps_Per_Day_post_missing'].transform('mean')



In [ ]:
df_merge = pd.merge(b, a, left_on=['lastName'],right_on=['study_id'])
df_merge

In [ ]:
df_valid_days_daycount['User UUID'].nunique()

In [ ]:
df_valid_days_daycount.groupby( 'User UUID')['Start_Time_Date_Offset_x'].transform('nunique').mean()

In [ ]:
df_valid_days_daycount_dup = df_valid_days_daycount.drop_duplicates(subset=['User UUID', 'Average_Monthly_Steps_Lockdown_post_missing','Month_Year_Name'], keep='first')
df_valid_days_daycount_dup

In [ ]:
df_valid_days_daycount_dup['User UUID'].nunique()

In [ ]:
df_valid_days_daycount_dup['Month_Year_Name'].unique()

In [ ]:
#Min
df_valid_days_daycount_dup.groupby('closure_status')['Average_Monthly_Steps_Lockdown_post_missing'].min()


In [ ]:

#Max
df_valid_days_daycount_dup.groupby('closure_status')['Average_Monthly_Steps_Lockdown_post_missing'].max()


In [ ]:
#Mean
df_valid_days_daycount_dup.groupby('closure_status')['Average_Monthly_Steps_Lockdown_post_missing'].mean()


In [ ]:

#Std
df_valid_days_daycount_dup.groupby('closure_status')['Average_Monthly_Steps_Lockdown_post_missing'].std()


In [ ]:

#Users
df_valid_days_daycount_dup.groupby('closure_status')['User UUID'].nunique()

In [ ]:
df_valid_days_daycount_dup['Month_Year_Name'].unique()


In [ ]:
df_merge_demo = pd.merge(b, a, left_on=['lastName'],right_on=['study_id'])

In [ ]:
df_closure_demo = pd.merge(df_merge_demo, df_valid_days_daycount_dup, left_on=['uuid'],right_on=['User UUID'])

In [ ]:
df_closure_demo = df_closure_demo.drop_duplicates(subset=['User UUID','closure_status'], keep='first')

In [ ]:
df_closure_demo.groupby('closure_status')['sex'].value_counts()

In [ ]:
df_closure_demo[df_closure_demo['closure_status']=='During Closure']['sex'].value_counts()

In [ ]:
df_closure_demo.groupby('closure_status')['trt'].value_counts()

In [ ]:
df_closure_demo.groupby('closure_status')['sex'].value_counts()

In [ ]:
df_closure_demo.groupby('closure_status')['age'].median()

In [ ]:
df_closure_demo.groupby('closure_status')['age'].std()

In [ ]:
df_closure_demo[(df_closure_demo['age']<11)&(df_closure_demo['age']>=5)].groupby('closure_status')['uuid'].nunique()

In [ ]:
df_closure_demo[(df_closure_demo['age']<14)&(df_closure_demo['age']>=11)].groupby('closure_status')['uuid'].nunique()

In [ ]:
df_closure_demo[(df_closure_demo['age']<=18)&(df_closure_demo['age']>=14)].groupby('closure_status')['uuid'].nunique()

In [ ]:
df_closure_demo.groupby('closure_status')['race'].value_counts()

In [ ]:
df_closure_demo.groupby('closure_status')['ethnic'].value_counts()


In [ ]:
#new: One average per month

df_valid_days_daycount_dup_month=df_valid_days_daycount_dup

df_valid_days_daycount_dup_month['average_steps_per_month_post_missing']=df_valid_days_daycount_dup_month.groupby(['Month_Year_Name'])['Average_Monthly_Steps_Lockdown_post_missing'].transform('mean')

df_valid_days_daycount_dup_month = df_valid_days_daycount_dup_month.drop_duplicates(subset=['average_steps_per_month_post_missing','Month_Year_Name'], keep='first')

In [ ]:
df_valid_days_daycount_dup_month['Month_Year_Name'].unique()


In [ ]:

#Min
df_valid_days_daycount_dup_month.groupby('closure_status')['average_steps_per_month_post_missing'].min()



In [ ]:


#Max
df_valid_days_daycount_dup_month.groupby('closure_status')['average_steps_per_month_post_missing'].max()


In [ ]:

#Mean
df_valid_days_daycount_dup_month.groupby('closure_status')['average_steps_per_month_post_missing'].mean()



In [ ]:


#Std
df_valid_days_daycount_dup_month.groupby('closure_status')['average_steps_per_month_post_missing'].std()


In [ ]:
#Users
df_valid_days_daycount_dup_month.groupby('closure_status')['User UUID'].nunique()

In [ ]:
#Exclude summer for TOD comparison with overall

df_valid_days_daycount_dup_month_non_sum=df_valid_days_daycount_dup_month[(df_valid_days_daycount_dup_month['Month_Year_Name']!='Jun-2020')
                                                                       &(df_valid_days_daycount_dup_month['Month_Year_Name']!='Jul-2020')
                                                                       &(df_valid_days_daycount_dup_month['Month_Year_Name']!='Aug-2020')
                                                               &(df_valid_days_daycount_dup_month['Month_Year_Name']!='Jun-2019')
                                                               &(df_valid_days_daycount_dup_month['Month_Year_Name']!='Jul-2019')
                                                               &(df_valid_days_daycount_dup_month['Month_Year_Name']!='Aug-2019')
                                                                &(df_valid_days_daycount_dup_month['Month_Year_Name']!='Jun-2021')
                                                               &(df_valid_days_daycount_dup_month['Month_Year_Name']!='Jul-2021')
                                                               &(df_valid_days_daycount_dup_month['Month_Year_Name']!='Aug-2021')]



In [ ]:
#Mean
df_valid_days_daycount_dup_month_non_sum.groupby('closure_status')['average_steps_per_month_post_missing'].mean()



In [ ]:
#Figure

df_valid_days_daycount_dup_fig=df_valid_days_daycount_dup
df_valid_days_daycount_dup_fig['pre_avg']= 8809.83


plt.figure(figsize=(60,20))
sns.set(font_scale =5)
sns.set_style("white")
sns.set_palette("Set1", 8, .75)
ax=sns.lineplot(x = 'Month_Year_Name', y = 'Average_Monthly_Steps_Lockdown_post_missing'
                , data = df_valid_days_daycount_dup_fig
             ,color='darkcyan'
             ,linewidth = 15
           , sort=False)
sns.lineplot(x = 'Month_Year_Name', y = 'pre_avg'
                , data = df_valid_days_daycount_dup_fig
             ,color='grey'
            , linestyle='--'
             ,linewidth = 10
           , sort=False)

plt.setp(ax.get_xticklabels(), rotation=90)
ax.set_xlabel("Month",fontdict= { 'fontsize': 60, 'fontweight':'bold'}, y =2)
ax.set_ylabel("Average Daily Number of Steps Per Month",fontdict= { 'fontsize': 55, 'fontweight':'bold'})
ax.set_title('Average Daily Step Values for Participants (Non zero MMI, after missingness analysis)',fontdict= { 'fontsize': 60, 'fontweight':'bold'}, y =1.1)
ax.axvline("1-14 Mar-2020", color="firebrick", linestyle="--", lw=10)
plt.legend(loc="upper right", frameon=True, fontsize=50)
plt.setp(ax.get_legend().get_texts(), fontsize='50') # for legend text
plt.setp(ax.get_legend().get_title(), fontsize='50') # for legend title


In [ ]:
#Statistical test

steps_pre_closure=df_valid_days_daycount_dup[df_valid_days_daycount_dup['closure_status']=='Pre Closure']

steps_pre_closure['Monthly_Avg']=steps_pre_closure.groupby(['Month_Year_Name'])['Average_Monthly_Steps_Lockdown_post_missing'].transform('mean')

steps_pre_closure['month_num']=steps_pre_closure['Start_Time_Offset'].dt.month
#steps_pre['month_num'].unique()
steps_pre_closure=steps_pre_closure.sort_values(by=['month_num'])
steps_pre_closure

steps_pre_closure_col=steps_pre_closure[['Month_Year_Name','Monthly_Avg']]
steps_pre_closure_dup = steps_pre_closure_col.drop_duplicates(subset=['Monthly_Avg', 'Month_Year_Name'], keep='first')
steps_pre_closure_dup

In [ ]:
steps_dur_closure=df_valid_days_daycount_dup[df_valid_days_daycount_dup['closure_status']=='During Closure']

steps_dur_closure['Monthly_Avg']=steps_dur_closure.groupby(['Month_Year_Name'])['Average_Monthly_Steps_Lockdown_post_missing'].transform('mean')

steps_dur_closure['month_num']=steps_dur_closure['Start_Time_Offset'].dt.month
steps_dur_closure=steps_dur_closure.sort_values(by=['month_num'])
steps_dur_closure

steps_dur_closure_col=steps_dur_closure[['Month_Year_Name','Monthly_Avg']]
steps_dur_closure_dup = steps_dur_closure_col.drop_duplicates(subset=['Monthly_Avg', 'Month_Year_Name'], keep='first')
steps_dur_closure_dup

In [ ]:
from scipy.stats import mannwhitneyu
U1, p = mannwhitneyu(steps_dur_closure_dup['Monthly_Avg'], steps_pre_closure_dup['Monthly_Avg'])
print(U1)
print(p*100)

print(p)

In [ ]:
#Time of day analysis

df_closure_tod=df_valid_days_daycount

df_closure_tod['Start Time Only Time']=df_closure_tod['Start_Time_Offset'].dt.strftime("%H:%M:%S")
# all_data_merged_steps[all_data_merged_steps['Start Time Only Time']>'16:15:00']
# base on data instead
df_closure_tod['Time_Of_Day']="Night"
df_closure_tod.loc[(df_closure_tod['Start Time Only Time'] <'11:00:00') & (df_closure_tod['Start Time Only Time'] >='07:00:00'), 'Time_Of_Day'] = 'Morning'
df_closure_tod.loc[(df_closure_tod['Start Time Only Time'] <'16:00:00') & (df_closure_tod['Start Time Only Time'] >='11:00:00'), 'Time_Of_Day'] = 'Afternoon'
df_closure_tod.loc[(df_closure_tod['Start Time Only Time'] <'20:00:00') & (df_closure_tod['Start Time Only Time'] >='16:00:00'), 'Time_Of_Day'] = 'Evening'



In [ ]:

df_closure_tod['Wkday_Num']=df_closure_tod['Start_Time_Offset'].dt.weekday

df_closure_tod['Wkday_Num'].unique()



In [ ]:
df_closure_tod_wkday=df_closure_tod[df_closure_tod['Wkday_Num'].isin([0,1,2,3,4])]
df_closure_tod_wkday['Wkday_Num'].unique()



In [ ]:
df_closure_tod_wkday[['Wkday_Num','Start_Time_Offset']]

In [ ]:
df_closure_tod_wkday_school=df_closure_tod_wkday[(df_closure_tod_wkday['Time_Of_Day']=='Morning')
                                                   |(df_closure_tod_wkday['Time_Of_Day']=='Afternoon')]


In [ ]:

df_closure_tod_wkday_school_non_sum=df_closure_tod_wkday_school[(df_closure_tod_wkday_school['Mon_Year']!='Jun-2020')
                                                                       &(df_closure_tod_wkday_school['Mon_Year']!='Jul-2020')
                                                                       &(df_closure_tod_wkday_school['Mon_Year']!='Aug-2020')
                                                               &(df_closure_tod_wkday_school['Mon_Year']!='Jun-2019')
                                                               &(df_closure_tod_wkday_school['Mon_Year']!='Jul-2019')
                                                               &(df_closure_tod_wkday_school['Mon_Year']!='Aug-2019')
                                                                &(df_closure_tod_wkday_school['Mon_Year']!='Jun-2021')
                                                               &(df_closure_tod_wkday_school['Mon_Year']!='Jul-2021')
                                                               &(df_closure_tod_wkday_school['Mon_Year']!='Aug-2021')]



In [ ]:
df_closure_tod_wkday_school_non_sum['Daily_Sum_TOD']=df_closure_tod_wkday_school_non_sum.groupby(['Start_Time_Date_Offset_x', 'User UUID'])['Value_x'].transform('sum')


df_closure_tod_wkday_school_non_sum['mean_monthly_steps_school']=df_closure_tod_wkday_school_non_sum.groupby([ 'User UUID','Month_Year_Name'])['Daily_Sum_TOD'].transform('mean')



In [ ]:
df_closure_tod_wkday_school_non_sum_dup = df_closure_tod_wkday_school_non_sum.drop_duplicates(subset=['User UUID', 'mean_monthly_steps_school','Month_Year_Name'], keep='first')

In [ ]:
#Min
df_closure_tod_wkday_school_non_sum_dup.groupby('closure_status')['mean_monthly_steps_school'].min()


In [ ]:
#Max
df_closure_tod_wkday_school_non_sum_dup.groupby('closure_status')['mean_monthly_steps_school'].max()


In [ ]:

#Mean
df_closure_tod_wkday_school_non_sum_dup.groupby('closure_status')['mean_monthly_steps_school'].mean()

In [ ]:

#Std
df_closure_tod_wkday_school_non_sum_dup.groupby('closure_status')['mean_monthly_steps_school'].std()


In [ ]:

#Users
df_closure_tod_wkday_school_non_sum_dup.groupby('closure_status')['User UUID'].nunique()

In [ ]:
sns.__version__

In [ ]:
df_closure_tod_fig=df_closure_tod_wkday_school_non_sum_dup[df_closure_tod_wkday_school_non_sum_dup['closure_status']!='Post Closure']

plt.figure(figsize=(100,40))
sns.set_style("white")
palette = {"Pre Closure":"mediumpurple",
            "During Closure":"lightcoral"
           #2020:"salmon"
           }
ax = sns.boxplot(x="Month_Year_Name",y="mean_monthly_steps_school", palette=palette,hue='closure_status',
                 data=df_closure_tod_fig, linewidth=14,
                showfliers=False,
            showmeans=True,dodge=False,width=0.5)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
sns.set(font_scale =8)

ax.set_xlabel('Month', fontdict= { 'fontsize':90, 'fontweight':'bold'} )
ax.set_ylabel('Number of Steps', fontdict= { 'fontsize':90, 'fontweight':'bold'} )

ax.set_title('Average Daily Step Count Values for Participants Before and After Lockdown During School Times', 
                  fontdict= { 'fontsize':90, 'fontweight':'bold'}, y =1.05)
plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0,frameon=True,edgecolor='black'
           ,title='Pandemic Status',framealpha=0.5)

In [ ]:

df_closure_tod_fig=df_closure_tod_wkday_school_non_sum_dup[df_closure_tod_wkday_school_non_sum_dup['closure_status']!='Post Closure']
fig_2_df_closure_tod=df_closure_tod_fig

fig_2_df_closure_tod['Month_Year_Name_Closure']=fig_2_df_closure_tod['Month_Year_Name'].str[0:3]
fig_2_df_closure_tod.loc[(fig_2_df_closure_tod['Month_Year_Name']=='1-14 Mar-2020'), 'Month_Year_Name_Closure']='Mar'
fig_2_df_closure_tod.loc[(fig_2_df_closure_tod['Month_Year_Name']=='15-31 Mar-2020'), 'Month_Year_Name_Closure']='Mar'
fig_2_df_closure_tod['month_only'] = pd.Categorical(fig_2_df_closure_tod['Month_Year_Name_Closure'],
                                   categories=['Jan', 'Feb','Mar', 'Apr', 'May', 'Sep', 'Oct', 'Nov', 'Dec'],ordered=True)



In [ ]:

plt.figure(figsize=(100,40))
sns.set_style("white")
palette = {"Pre Closure":"mediumpurple",
            "During Closure":"lightcoral"
           }
ax = sns.boxplot(x="month_only",y="mean_monthly_steps_school", palette=palette,hue='closure_status',
                 data=fig_2_df_closure_tod, linewidth=14,
                showfliers=False,
            showmeans=False,width=0.5)

ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
sns.set(font_scale =8)

ax.set_xlabel('Month', fontdict= { 'fontsize':90, 'fontweight':'bold'} )
ax.set_ylabel('Number of Steps', fontdict= { 'fontsize':90, 'fontweight':'bold'} )

ax.set_title('Average Daily Step Count Values for Participants Before and After Lockdown During School Times', 
                  fontdict= { 'fontsize':90, 'fontweight':'bold'}, y =1.05)
plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0,frameon=True,edgecolor='black'
           ,title='Pandemic Status',framealpha=0.5)

In [ ]:
#New one value per month

df_closure_tod_wkday_school_non_sum_dup_month=df_closure_tod_wkday_school_non_sum_dup

df_closure_tod_wkday_school_non_sum_dup_month['mean_monthly_steps_school_per_month']=df_closure_tod_wkday_school_non_sum_dup_month.groupby(['Month_Year_Name'])['mean_monthly_steps_school'].transform('mean')



df_closure_tod_wkday_school_non_sum_dup_month_dup = df_closure_tod_wkday_school_non_sum_dup_month.drop_duplicates(subset=['mean_monthly_steps_school_per_month','Month_Year_Name'], keep='first')

In [ ]:
#Min
df_closure_tod_wkday_school_non_sum_dup_month_dup.groupby('closure_status')['mean_monthly_steps_school_per_month'].min()


In [ ]:
#Max
df_closure_tod_wkday_school_non_sum_dup_month_dup.groupby('closure_status')['mean_monthly_steps_school_per_month'].max()


In [ ]:

#Mean
df_closure_tod_wkday_school_non_sum_dup_month_dup.groupby('closure_status')['mean_monthly_steps_school_per_month'].mean()

In [ ]:

#Std
df_closure_tod_wkday_school_non_sum_dup_month_dup.groupby('closure_status')['mean_monthly_steps_school_per_month'].std()


In [ ]:

#Users
df_closure_tod_wkday_school_non_sum_dup_month_dup.groupby('closure_status')['User UUID'].nunique()

In [ ]:
#statistical analysis for in-school non-summer months

In [ ]:
#Statistical test

steps_pre_closure_wkday_school_non_sum=df_closure_tod_wkday_school_non_sum_dup[df_closure_tod_wkday_school_non_sum_dup['closure_status']=='Pre Closure']

steps_pre_closure_wkday_school_non_sum['Monthly_Avg']=steps_pre_closure_wkday_school_non_sum.groupby(['Month_Year_Name'])['mean_monthly_steps_school'].transform('mean')

steps_pre_closure_wkday_school_non_sum['month_num']=steps_pre_closure_wkday_school_non_sum['Start_Time_Offset'].dt.month

steps_pre_closure_wkday_school_non_sum=steps_pre_closure_wkday_school_non_sum.sort_values(by=['month_num'])
steps_pre_closure_wkday_school_non_sum

steps_pre_closure_wkday_school_non_sum_col=steps_pre_closure_wkday_school_non_sum[['Month_Year_Name','Monthly_Avg']]
steps_pre_closure_wkday_school_non_sum_col_dup = steps_pre_closure_wkday_school_non_sum_col.drop_duplicates(subset=['Monthly_Avg', 'Month_Year_Name'], keep='first')
steps_pre_closure_wkday_school_non_sum_col_dup

In [ ]:
steps_dur_closure_wkday_school_non_sum=df_closure_tod_wkday_school_non_sum_dup[df_closure_tod_wkday_school_non_sum_dup['closure_status']=='During Closure']

steps_dur_closure_wkday_school_non_sum['Monthly_Avg']=steps_dur_closure_wkday_school_non_sum.groupby(['Month_Year_Name'])['mean_monthly_steps_school'].transform('mean')

steps_dur_closure_wkday_school_non_sum['month_num']=steps_dur_closure_wkday_school_non_sum['Start_Time_Offset'].dt.month

steps_dur_closure_wkday_school_non_sum=steps_dur_closure_wkday_school_non_sum.sort_values(by=['month_num'])
steps_dur_closure_wkday_school_non_sum

steps_dur_closure_wkday_school_non_sum_col=steps_dur_closure_wkday_school_non_sum[['Month_Year_Name','Monthly_Avg']]
steps_dur_closure_wkday_school_non_sum_col_dup = steps_dur_closure_wkday_school_non_sum_col.drop_duplicates(subset=['Monthly_Avg', 'Month_Year_Name'], keep='first')
steps_dur_closure_wkday_school_non_sum_col_dup

In [ ]:
from scipy.stats import mannwhitneyu
U1, p = mannwhitneyu(steps_dur_closure_wkday_school_non_sum_col_dup['Monthly_Avg'], steps_pre_closure_wkday_school_non_sum_col_dup['Monthly_Avg'])
print(U1)
print(p*100)

print(p)

In [ ]:
#Statistical test (exact copy of previous test-redundant; made changes to the code cells above)

steps_pre_closure=df_valid_days_daycount_dup[df_valid_days_daycount_dup['closure_status']=='Pre Closure']

steps_pre_closure['Monthly_Avg']=steps_pre_closure.groupby(['Month_Year_Name'])['Average_Monthly_Steps_Lockdown_post_missing'].transform('mean')

steps_pre_closure['month_num']=steps_pre_closure['Start_Time_Offset'].dt.month
#steps_pre['month_num'].unique()
steps_pre_closure=steps_pre_closure.sort_values(by=['month_num'])
steps_pre_closure

steps_pre_closure_col=steps_pre_closure[['Month_Year_Name','Monthly_Avg']]
steps_pre_closure_dup = steps_pre_closure_col.drop_duplicates(subset=['Monthly_Avg', 'Month_Year_Name'], keep='first')
steps_pre_closure_dup

steps_dur_closure=df_valid_days_daycount_dup[df_valid_days_daycount_dup['closure_status']=='During Closure']

steps_dur_closure['Monthly_Avg']=steps_dur_closure.groupby(['Month_Year_Name'])['Average_Monthly_Steps_Lockdown_post_missing'].transform('mean')

steps_dur_closure['month_num']=steps_dur_closure['Start_Time_Offset'].dt.month
#steps_pre['month_num'].unique()
steps_dur_closure=steps_dur_closure.sort_values(by=['month_num'])
steps_dur_closure

steps_dur_closure_col=steps_dur_closure[['Month_Year_Name','Monthly_Avg']]
steps_dur_closure_dup = steps_dur_closure_col.drop_duplicates(subset=['Monthly_Avg', 'Month_Year_Name'], keep='first')
steps_dur_closure_dup

from scipy.stats import mannwhitneyu
U1, p = mannwhitneyu(steps_dur_closure_dup['Monthly_Avg'], steps_pre_closure_dup['Monthly_Avg'])
print(U1)
print(p*100)

print(p)

In [ ]:
#Summer months weekday steps analysis (pre-, during-, and post-closure)

In [ ]:
df_closure_tod_wkday_school_only=df_closure_tod_wkday_school

In [ ]:

df_closure_tod_wkday_school_only_sum=df_closure_tod_wkday_school[(df_closure_tod_wkday_school['Mon_Year']=='Jun-2020')
                                                                       |(df_closure_tod_wkday_school['Mon_Year']=='Jul-2020')
                                                                       |(df_closure_tod_wkday_school['Mon_Year']=='Aug-2020')
                                                               |(df_closure_tod_wkday_school['Mon_Year']=='Jun-2019')
                                                               |(df_closure_tod_wkday_school['Mon_Year']=='Jul-2019')
                                                               |(df_closure_tod_wkday_school['Mon_Year']=='Aug-2019')
                                                                |(df_closure_tod_wkday_school['Mon_Year']=='Jun-2021')
                                                               |(df_closure_tod_wkday_school['Mon_Year']=='Jul-2021')
                                                               |(df_closure_tod_wkday_school['Mon_Year']=='Aug-2021')]


In [ ]:
df_closure_tod_wkday_school_only_sum['Daily_Sum_TOD']=df_closure_tod_wkday_school_only_sum.groupby(['Start_Time_Date_Offset_x', 'User UUID'])['Value_x'].transform('sum')


df_closure_tod_wkday_school_only_sum['mean_monthly_steps_school_sum']=df_closure_tod_wkday_school_only_sum.groupby([ 'User UUID','Month_Year_Name'])['Daily_Sum_TOD'].transform('mean')



In [ ]:
df_closure_tod_wkday_school_only_sum_dup = df_closure_tod_wkday_school_only_sum.drop_duplicates(subset=['User UUID', 'mean_monthly_steps_school_sum','Month_Year_Name'], keep='first')
df_closure_tod_wkday_school_only_sum_dup

In [ ]:
#Min
df_closure_tod_wkday_school_only_sum_dup.groupby('closure_status')['mean_monthly_steps_school_sum'].min()


In [ ]:
#Max
df_closure_tod_wkday_school_only_sum_dup.groupby('closure_status')['mean_monthly_steps_school_sum'].max()


In [ ]:

#Mean
df_closure_tod_wkday_school_only_sum_dup.groupby('closure_status')['mean_monthly_steps_school_sum'].mean()

In [ ]:

#Std
df_closure_tod_wkday_school_only_sum_dup.groupby('closure_status')['mean_monthly_steps_school_sum'].std()


In [ ]:

#Users
df_closure_tod_wkday_school_only_sum_dup.groupby('closure_status')['User UUID'].nunique()

In [ ]:
df_closure_tod_wkday_school_only_sum_dup_fig=df_closure_tod_wkday_school_only_sum_dup[df_closure_tod_wkday_school_only_sum_dup['closure_status']!='Post Closure']

plt.figure(figsize=(100,40))
sns.set_style("white")
palette = {"Pre Closure":"mediumpurple",
            "During Closure":"lightcoral"
           #2020:"salmon"
           }
ax = sns.boxplot(x="Month_Year_Name",y="mean_monthly_steps_school_sum", palette=palette,hue='closure_status',
                 data=df_closure_tod_wkday_school_only_sum_dup_fig, linewidth=14,
                showfliers=False,
            showmeans=True,dodge=False,width=0.5)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
#ax.set_title('Average Step values for Participants Before and After Lockdown For School Times')
#ax.set(xlabel='Month', ylabel='Number of Steps')
sns.set(font_scale =8)

ax.set_xlabel('Month', fontdict= { 'fontsize':90, 'fontweight':'bold'} )
ax.set_ylabel('Number of Steps', fontdict= { 'fontsize':90, 'fontweight':'bold'} )

ax.set_title('Average Daily Step Count Values for Participants Before and After Lockdown During School Times', 
                  fontdict= { 'fontsize':90, 'fontweight':'bold'}, y =1.05)
plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0,frameon=True,edgecolor='black'
           ,title='Pandemic Status',framealpha=0.5)

In [ ]:

df_closure_tod_wkday_school_only_sum_dup_fig=df_closure_tod_wkday_school_only_sum_dup[df_closure_tod_wkday_school_only_sum_dup['closure_status']!='Post Closure']
fig_3_df_closure_tod=df_closure_tod_wkday_school_only_sum_dup_fig

fig_3_df_closure_tod['Month_Year_Name_Closure']=fig_3_df_closure_tod['Month_Year_Name'].str[0:3]
fig_3_df_closure_tod['month_only'] = pd.Categorical(fig_3_df_closure_tod['Month_Year_Name_Closure'],
                                   categories=['Jun', 'Jul', 'Aug'],ordered=True)



In [ ]:

plt.figure(figsize=(100,40))
sns.set_style("white")
palette = {"Pre Closure":"yellowgreen",
            "During Closure":"lightsalmon"
           #2020:"salmon"
           }
ax = sns.boxplot(x="month_only",y="mean_monthly_steps_school_sum", palette=palette,hue='closure_status',
                 data=fig_3_df_closure_tod, linewidth=14,
                showfliers=False,
            showmeans=False,width=0.5)

ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
#ax.set_title('Average Step values for Participants Before and After Lockdown For School Times')
#ax.set(xlabel='Month', ylabel='Number of Steps')
sns.set(font_scale =8)

ax.set_xlabel('Month', fontdict= { 'fontsize':90, 'fontweight':'bold'} )
ax.set_ylabel('Number of Steps', fontdict= { 'fontsize':90, 'fontweight':'bold'} )

ax.set_title('Average Daily Step Count Values for Participants Before and After Lockdown During School Times', 
                  fontdict= { 'fontsize':90, 'fontweight':'bold'}, y =1.05)
plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0,frameon=True,edgecolor='black'
           ,title='Pandemic Status',framealpha=0.5)

In [ ]:
#New one value per month

df_closure_tod_wkday_school_only_sum_dup_month=df_closure_tod_wkday_school_only_sum_dup

df_closure_tod_wkday_school_only_sum_dup_month['mean_monthly_steps_school_sum_per_month']=df_closure_tod_wkday_school_only_sum_dup_month.groupby(['Month_Year_Name'])['mean_monthly_steps_school_sum'].transform('mean')



df_closure_tod_wkday_school_only_sum_dup_month_dup = df_closure_tod_wkday_school_only_sum_dup_month.drop_duplicates(subset=['mean_monthly_steps_school_sum_per_month','Month_Year_Name'], keep='first')
df_closure_tod_wkday_school_only_sum_dup_month_dup

In [ ]:
#Min
df_closure_tod_wkday_school_only_sum_dup_month_dup.groupby('closure_status')['mean_monthly_steps_school_sum_per_month'].min()


In [ ]:
#Max
df_closure_tod_wkday_school_only_sum_dup_month_dup.groupby('closure_status')['mean_monthly_steps_school_sum_per_month'].max()


In [ ]:

#Mean
df_closure_tod_wkday_school_only_sum_dup_month_dup.groupby('closure_status')['mean_monthly_steps_school_sum_per_month'].mean()

In [ ]:

#Std
df_closure_tod_wkday_school_only_sum_dup_month_dup.groupby('closure_status')['mean_monthly_steps_school_sum_per_month'].std()


In [ ]:

#Users
df_closure_tod_wkday_school_only_sum_dup_month_dup.groupby('closure_status')['User UUID'].nunique()

In [ ]:
#Statistical test for summer weekdays pre and post closure

In [ ]:
df_closure_tod_wkday_school_only_sum_dup

In [ ]:
#Statistical test

steps_pre_closure_wkday_school_only_sum=df_closure_tod_wkday_school_only_sum_dup[df_closure_tod_wkday_school_only_sum_dup['closure_status']=='Pre Closure']

steps_pre_closure_wkday_school_only_sum['Monthly_Avg']=steps_pre_closure_wkday_school_only_sum.groupby(['Month_Year_Name'])['mean_monthly_steps_school_sum'].transform('mean')

steps_pre_closure_wkday_school_only_sum['month_num']=steps_pre_closure_wkday_school_only_sum['Start_Time_Offset'].dt.month

steps_pre_closure_wkday_school_only_sum=steps_pre_closure_wkday_school_only_sum.sort_values(by=['month_num'])


steps_pre_closure_wkday_school_only_sum_col=steps_pre_closure_wkday_school_only_sum[['Month_Year_Name','Monthly_Avg']]
steps_pre_closure_wkday_school_only_sum_col_dup = steps_pre_closure_wkday_school_only_sum_col.drop_duplicates(subset=['Monthly_Avg', 'Month_Year_Name'], keep='first')
steps_pre_closure_wkday_school_only_sum_col_dup

In [ ]:
steps_dur_closure_wkday_school_only_sum=df_closure_tod_wkday_school_only_sum_dup[df_closure_tod_wkday_school_only_sum_dup['closure_status']=='During Closure']

steps_dur_closure_wkday_school_only_sum['Monthly_Avg']=steps_dur_closure_wkday_school_only_sum.groupby(['Month_Year_Name'])['mean_monthly_steps_school_sum'].transform('mean')

steps_dur_closure_wkday_school_only_sum['month_num']=steps_dur_closure_wkday_school_only_sum['Start_Time_Offset'].dt.month

steps_dur_closure_wkday_school_only_sum=steps_dur_closure_wkday_school_only_sum.sort_values(by=['month_num'])
steps_dur_closure_wkday_school_only_sum

steps_dur_closure_wkday_school_only_sum_col=steps_dur_closure_wkday_school_only_sum[['Month_Year_Name','Monthly_Avg']]
steps_dur_closure_wkday_school_only_sum_col_dup = steps_dur_closure_wkday_school_only_sum_col.drop_duplicates(subset=['Monthly_Avg', 'Month_Year_Name'], keep='first')
steps_dur_closure_wkday_school_only_sum_col_dup

In [ ]:
from scipy.stats import mannwhitneyu
U1, p = mannwhitneyu(steps_dur_closure_wkday_school_only_sum_col_dup['Monthly_Avg'], steps_pre_closure_wkday_school_only_sum_col_dup['Monthly_Avg'])
print(U1)
print(p*100)

print(p)

In [ ]:
#Sleep

In [ ]:
df_sleep=df[df['Type']=='sleepDuration']
len(df_sleep)

In [ ]:
df_sleep['User UUID'].nunique()

In [ ]:
steps_user_id_list=df_valid_days_daycount['User UUID'].unique()

In [ ]:
df_sleep_step_ids=df_sleep[df_sleep['User UUID'].isin(steps_user_id_list)]
df_sleep_step_ids

In [ ]:
df_sleep_step_ids['Start Time']=df_sleep_step_ids['Start Time'].apply(pd.to_datetime)

df_sleep_step_ids['End Time']=df_sleep_step_ids['End Time'].apply(pd.to_datetime)

df_sleep_step_ids['Start_Time_Offset']=df_sleep_step_ids['Start Time']+ pd.to_timedelta(df_sleep_step_ids['Time Zone Offset'], unit='s')
df_sleep_step_ids['End_Time_Offset']=df_sleep_step_ids['End Time']+ pd.to_timedelta(df_sleep_step_ids['Time Zone Offset'], unit='s')

df_sleep_step_ids['Start_Time_Date_Offset'] = df_sleep_step_ids['Start_Time_Offset'].dt.date
df_sleep_step_ids['End_Time_Date_Offset'] = df_sleep_step_ids['End_Time_Offset'].dt.date
df_sleep_step_ids['month_year'] = df_sleep_step_ids['Start_Time_Offset'].dt.to_period('M')

df_sleep_step_ids['Mon_Year'] = df_sleep_step_ids['Start_Time_Offset'].dt.strftime('%b-%Y')
df_sleep_step_ids['Month_Year_Name']=df_sleep_step_ids['Mon_Year']

In [ ]:
df_sleep_copy=df_sleep_step_ids

df_sleep_copy_dup = df_sleep_copy.drop_duplicates(subset=['User UUID', 'Start Time','End Time','Value'], keep='first')


In [ ]:
df_sleep_copy_dup['year_only'] = df_sleep_copy_dup['Start_Time_Offset'].dt.year
df_sleep_copy_dup['month_only'] = df_sleep_copy_dup['Start_Time_Offset'].dt.strftime("%B")
df_sleep_copy_dup=df_sleep_copy_dup.sort_values(by=['Start_Time_Offset'])
df_sleep_copy_dup['Start Time Only Time']=df_sleep_copy_dup['Start_Time_Offset'].dt.strftime("%H:%M:%S")

In [ ]:
#convert to datetime format to calculate bedtime date
df_sleep_copy_dup['Start Time Only Time Datetime']=df_sleep_copy_dup['Start Time Only Time'].apply(pd.to_datetime)

In [ ]:
df_sleep_copy_dup['Start Time Only Time Datetime'].dt.date.unique()

In [ ]:
#Get only nighttime values
#change to today's date
df_sleep_bedtime_calc_allnight= df_sleep_copy_dup[(df_sleep_copy_dup['Start Time Only Time Datetime']>dt.datetime(2023, 6, 27, 18, 0, 0))|
                                         (df_sleep_copy_dup['Start Time Only Time Datetime']<dt.datetime(2023, 6, 27, 8, 0, 0))]

In [ ]:
df_sleep_bedtime_calc_allnight['Start Time Only Time Datetime'].dt.date.unique()

In [ ]:
df_sleep_bedtime_calc_allnight['Start Time Only Time Datetime']=df_sleep_bedtime_calc_allnight['Start Time Only Time'].apply(pd.to_datetime)

In [ ]:
from datetime import timedelta

In [ ]:
df_sleep_bedtime_calc_allnight_val=df_sleep_bedtime_calc_allnight

#change according to today's date
df_sleep_bedtime_calc_allnight_val.loc[(df_sleep_bedtime_calc_allnight_val['Start Time Only Time Datetime']>dt.datetime(2023, 6, 27, 18, 0, 0)), 'Bedtime_Date'] = df_sleep_bedtime_calc_allnight_val['Start_Time_Date_Offset']

df_sleep_bedtime_calc_allnight_val.loc[(df_sleep_bedtime_calc_allnight_val['Start Time Only Time Datetime']<dt.datetime(2023, 6, 27, 8, 0, 0)), 'Bedtime_Date'] = df_sleep_bedtime_calc_allnight_val['Start_Time_Date_Offset'] - timedelta(days=1)

In [ ]:
df_sleep_bedtime_calc_allnight_val['Start_Time_Offset_dt']=pd.to_datetime(df_sleep_bedtime_calc_allnight_val['Start_Time_Offset'])

df_sleep_bedtime_calc_allnight_val['End_Time_Offset_dt']=pd.to_datetime(df_sleep_bedtime_calc_allnight_val['End_Time_Offset'])


In [ ]:
df_sleep_bedtime_calc_allnight_val['Start_Time_Offset_dt_iso']=df_sleep_bedtime_calc_allnight_val['Start_Time_Offset_dt'].astype('int64')//1e9

df_sleep_bedtime_calc_allnight_val['End_Time_Offset_dt_iso']=df_sleep_bedtime_calc_allnight_val['End_Time_Offset_dt'].astype('int64')//1e9

df_sleep_bedtime_calc_allnight_val['Bedtime_Min_iso'] = df_sleep_bedtime_calc_allnight_val.groupby([ 'User UUID','Bedtime_Date'])['Start_Time_Offset_dt_iso'].transform('min')

df_sleep_bedtime_calc_allnight_val['Waketime_Max_iso'] = df_sleep_bedtime_calc_allnight_val.groupby([ 'User UUID','Bedtime_Date'])['End_Time_Offset_dt_iso'].transform('max')

df_sleep_bedtime_calc_allnight_val['Bedtime_Min_iso_int_converted']=pd.to_datetime(df_sleep_bedtime_calc_allnight_val['Bedtime_Min_iso'], unit='s')

df_sleep_bedtime_calc_allnight_val['Waketime_Max_iso_int_converted']=pd.to_datetime(df_sleep_bedtime_calc_allnight_val['Waketime_Max_iso'], unit='s')

df_sleep_bedtime_calc_allnight_val['Bedtime_Min_iso_int_converted_dt']=df_sleep_bedtime_calc_allnight_val['Bedtime_Min_iso_int_converted'].apply(pd.to_datetime)

df_sleep_bedtime_calc_allnight_val['Waketime_Max_iso_int_converted_dt']=df_sleep_bedtime_calc_allnight_val['Waketime_Max_iso_int_converted'].apply(pd.to_datetime)

df_sleep_bedtime_calc_allnight_val[['Bedtime_Min_iso_int_converted','Start_Time_Offset_dt_iso','Start Time Only Time','Start Time']]

df_sleep_bedtime_calc_allnight_val[['Waketime_Max_iso_int_converted_dt','End_Time_Offset_dt_iso','Start Time Only Time','End Time']]

df_sleep_bedtime_calc_allnight_val['Bedtime_Min_iso_int_converted_time']=df_sleep_bedtime_calc_allnight_val['Bedtime_Min_iso_int_converted'].dt.strftime("%H:%M:%S")


df_sleep_bedtime_calc_allnight_val['Waketime_Max_iso_int_converted_time']=df_sleep_bedtime_calc_allnight_val['Waketime_Max_iso_int_converted'].dt.strftime("%H:%M:%S")

In [ ]:
from cmath import rect, phase
from math import radians, degrees

def meanAngle(deg):
    complexDegree = sum(rect(1, radians(d)) for d in deg) / len(deg)
    argument = phase(complexDegree)
    meanAngle = degrees(argument)
    return meanAngle

def meanTime(times):
    t = (time.split(':') for time in times)
    seconds = ((float(s) + int(m) * 60 + int(h) * 3600) 
               for h, m, s in t)
    day = 24 * 60 * 60
    toAngles = [s * 360. / day for s in seconds]
    meanAsAngle = meanAngle(toAngles)
    meanSeconds = meanAsAngle * day / 360.
    if meanSeconds < 0:
        meanSeconds += day
    h, m = divmod(meanSeconds, 3600)
    m, s = divmod(m, 60)
    return('%02i:%02i:%02i' % (h, m, s))

In [ ]:
df_sleep_bedtime_calc_allnight_val_copy=df_sleep_bedtime_calc_allnight_val

In [ ]:
#get month from bedtime
df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date_month_year']=pd.to_datetime(df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date']).dt.to_period('M')
df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date_month_year_abb']=pd.to_datetime(df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date']).dt.strftime('%b')

In [ ]:
df_sleep_bedtime_calc_allnight_val_copy['Month_Year_Name_Bedtime']=df_sleep_bedtime_calc_allnight_val_copy['Mon_Year_Bedtime']

df_sleep_bedtime_calc_allnight_val_copy.loc[(df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date'] <dt.date(2020,3,15)) & (df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date'] >=dt.date(2020,3,1)), 'Month_Year_Name_Bedtime'] = '1-14 Mar-2020'
df_sleep_bedtime_calc_allnight_val_copy.loc[(df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date'] <dt.date(2020,4,1)) & (df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date'] >dt.date(2020,3,14)), 'Month_Year_Name_Bedtime'] = '15-31 Mar-2020'
df_sleep_bedtime_calc_allnight_val_copy.loc[(df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date'] >=dt.date(2020,4,1)), 'Month_Year_Name_Bedtime'] = df_sleep_bedtime_calc_allnight_val_copy.Month_Year_Name_Bedtime

In [ ]:
#new based on bedtime date month
column_Names_bt=df_sleep_bedtime_calc_allnight_val_copy.columns.values
df_sleep_bedtime_calc_allnight_val_mean = pd.DataFrame(columns=column_Names_bt)

unique_id_avg = df_sleep_bedtime_calc_allnight_val_copy['User UUID'].unique()

for i in unique_id_avg: 
    df_id_avg=df_sleep_bedtime_calc_allnight_val_copy[df_sleep_bedtime_calc_allnight_val_copy['User UUID']==i]
    print(i)
    
    unique_month = df_id_avg['Month_Year_Name_Bedtime'].unique()
        
    for j in unique_month:
        df_id_date_avg=df_id_avg[df_id_avg['Month_Year_Name_Bedtime']==j]
        df_id_date_avg['Average_Bedtime']=meanTime(df_id_date_avg['Bedtime_Min_iso_int_converted_time'])
        
        df_id_date_avg['Average_Waketime']=meanTime(df_id_date_avg['Waketime_Max_iso_int_converted_time'])
        
        df_sleep_bedtime_calc_allnight_val_mean=df_sleep_bedtime_calc_allnight_val_mean.append(df_id_date_avg)

In [ ]:
#restrict time
df_sleep_bedtime_calc_allnight_val_mean_copy=df_sleep_bedtime_calc_allnight_val_mean
df_sleep_bedtime_calc_allnight_val_mean=df_sleep_bedtime_calc_allnight_val_mean[(df_sleep_bedtime_calc_allnight_val_mean['Bedtime_Date']>=dt.date(2019,3,1)) & (df_sleep_bedtime_calc_allnight_val_mean['Bedtime_Date'] <=dt.date(2021,6,30))]

print(df_sleep_bedtime_calc_allnight_val_mean['Bedtime_Date'].min())

print(df_sleep_bedtime_calc_allnight_val_mean['Bedtime_Date'].max())

In [ ]:
df_sleep_bedtime_calc_allnight_val_mean.loc[(df_sleep_bedtime_calc_allnight_val_mean['Bedtime_Date'] <dt.date(2020,3,15)) & (df_sleep_bedtime_calc_allnight_val_mean['Bedtime_Date'] >=dt.date(2019,3,1)), 'closure_status'] = 'Pre Closure'

df_sleep_bedtime_calc_allnight_val_mean.loc[(df_sleep_bedtime_calc_allnight_val_mean['Bedtime_Date'] <=dt.date(2021,3,31)) & (df_sleep_bedtime_calc_allnight_val_mean['Bedtime_Date'] >=dt.date(2020,3,15)), 'closure_status'] = 'During Closure'

df_sleep_bedtime_calc_allnight_val_mean.loc[(df_sleep_bedtime_calc_allnight_val_mean['Bedtime_Date'] >dt.date(2021,3,31)), 'closure_status'] = 'Post Closure'

print(df_sleep_bedtime_calc_allnight_val_mean.groupby('closure_status')['Bedtime_Date'].min())

In [ ]:
df_sleep_bedtime_calc_allnight_val_mean_dup=df_sleep_bedtime_calc_allnight_val_mean.drop_duplicates(subset=['User UUID', 'Month_Year_Name_Bedtime'], keep='first')

In [ ]:
column_Names_avg=df_sleep_bedtime_calc_allnight_val_mean_dup.columns.values
df_avg_per_month = pd.DataFrame(columns=column_Names_avg)

month_avg = df_sleep_bedtime_calc_allnight_val_mean_dup['Month_Year_Name_Bedtime'].unique()
#j=0

for i in month_avg: 
    df_month_avg=df_sleep_bedtime_calc_allnight_val_mean_dup[df_sleep_bedtime_calc_allnight_val_mean_dup['Month_Year_Name_Bedtime']==i]
    df_month_avg['Average_Bedtime_per_period']=meanTime(df_month_avg['Average_Bedtime'])
    df_month_avg['Average_Waketime_per_period']=meanTime(df_month_avg['Average_Waketime'])
    #print(i)
    print(i)
    
    df_avg_per_month=df_avg_per_month.append(df_month_avg)

In [ ]:
df_avg_per_month_bt_dup = df_avg_per_month.drop_duplicates(subset=['Average_Bedtime_per_period','Month_Year_Name_Bedtime'], keep='first')


In [ ]:
df_avg_per_month_wt_dup = df_avg_per_month.drop_duplicates(subset=[ 'Average_Waketime_per_period','Month_Year_Name_Bedtime'], keep='first')

In [ ]:
#Calculate pre and post pandemic mean

Pre_pandemic_bt=df_avg_per_month_bt_dup[df_avg_per_month_bt_dup['closure_status']=='Pre Closure']
Pre_pandemic_wt=df_avg_per_month_wt_dup[df_avg_per_month_wt_dup['closure_status']=='Pre Closure']

#Calculate pre and post pandemic mean

Post_pandemic_bt=df_avg_per_month_wt_dup[df_avg_per_month_wt_dup['closure_status']=='During Closure']
Post_pandemic_wt=df_avg_per_month_wt_dup[df_avg_per_month_wt_dup['closure_status']=='During Closure']

meanTime(Pre_pandemic_bt['Average_Bedtime_per_period'])



In [ ]:
meanTime(Pre_pandemic_wt['Average_Waketime_per_period'])

In [ ]:
meanTime(Post_pandemic_wt['Average_Waketime_per_period'])

In [ ]:
def meanTime_a(times,type):
    t = (time.split(':') for time in times)
    seconds = ((float(s) + int(m) * 60 + int(h) * 3600) 
               for h, m, s in t)
    day = 24 * 60 * 60
    toAngles = [s * 360. / day for s in seconds]
    #x = [(ang + 360) if ang<120 else ang for ang in toAngles]
    x = [(ang + 360) if (ang<120 and type=='Night') else ang for ang in toAngles]
    mean = (np.mean(x) - 360) if np.mean(x) > 360 else np.mean(x)
    std = (np.std(x) - 360) if np.std(x) > 360 else np.std(x)
    print(x)
    print(mean)
    print(std)
    for val in [mean, std]:
        meanSeconds = val * day / 360.
        if meanSeconds < 0:
            meanSeconds += day
        h, m = divmod(meanSeconds, 3600)
        m, s = divmod(m, 60)
        print('%02i:%02i:%02i' % (h, m, s))


In [ ]:
Pre_pandemic_bt_sum=Pre_pandemic_bt[Pre_pandemic_bt['Month_Year_Name_Bedtime'].isin(['Jun-2019','Jul-2019','Aug-2019'])]

In [ ]:
meanTime(Pre_pandemic_bt_sum['Average_Bedtime_per_period'])

In [ ]:
meanTime_a(Pre_pandemic_bt_sum['Average_Bedtime_per_period'],'Night')

In [ ]:
Pre_pandemic_wt_sum=Pre_pandemic_wt[Pre_pandemic_wt['Month_Year_Name_Bedtime'].isin(['Jun-2019','Jul-2019','Aug-2019'])]
Pre_pandemic_wt_sum[['Average_Waketime_per_period','Month_Year_Name_Bedtime']]

In [ ]:
meanTime(Pre_pandemic_wt_sum['Average_Waketime_per_period'])

In [ ]:
meanTime_a(Pre_pandemic_wt_sum['Average_Waketime_per_period'],'Day')

In [ ]:
Post_pandemic_bt_sum=Post_pandemic_bt[Post_pandemic_bt['Month_Year_Name_Bedtime'].isin(['Jun-2020','Jul-2020','Aug-2020'])]


In [ ]:
meanTime_a(Post_pandemic_bt_sum['Average_Bedtime_per_period'],'Night')

In [ ]:
meanTime(Post_pandemic_bt_sum['Average_Bedtime_per_period'])

In [ ]:
Post_pandemic_wt_sum=Post_pandemic_wt[Post_pandemic_wt['Month_Year_Name_Bedtime'].isin(['Jun-2020','Jul-2020','Aug-2020'])]


In [ ]:
meanTime_a(Post_pandemic_wt_sum['Average_Waketime_per_period'],'Day')

In [ ]:
meanTime(Post_pandemic_wt_sum['Average_Waketime_per_period'])

In [ ]:
#Round average to nearest 15 min
df_avg_per_month_bt_dup=df_sleep_bedtime_calc_allnight_val_mean_dup
df_avg_per_month_bt_dup['Average_Bedtime_Round_fifteenmin']=pd.to_datetime(pd.to_datetime(pd.to_datetime(df_avg_per_month_bt_dup['Average_Bedtime'],format= '%H:%M:%S' ).dt.round('15min')).dt.strftime("%H:%M:%S"))

df_avg_per_month_wt_dup=df_sleep_bedtime_calc_allnight_val_mean_dup
df_avg_per_month_wt_dup['Average_Waketime_Round_fifteenmin']=pd.to_datetime(pd.to_datetime(pd.to_datetime(df_avg_per_month_wt_dup['Average_Waketime'],format= '%H:%M:%S' ).dt.round('15min')).dt.strftime("%H:%M:%S"))


In [ ]:
df_avg_per_month_bt_dup['bt_hr']=df_avg_per_month_bt_dup['Average_Bedtime_Round_fifteenmin'].dt.strftime("%H").astype(str)
df_avg_per_month_bt_dup['bt_min']=df_avg_per_month_bt_dup['Average_Bedtime_Round_fifteenmin'].dt.strftime("%M").astype(str)
df_avg_per_month_bt_dup['bt_float']=df_avg_per_month_bt_dup['bt_hr']+'.'+df_avg_per_month_bt_dup['bt_min']

In [ ]:
df_avg_per_month_bt_dup.loc[(df_avg_per_month_bt_dup['bt_min'] =='15'), 'bt_min_float'] = '25'
df_avg_per_month_bt_dup.loc[(df_avg_per_month_bt_dup['bt_min'] =='30'), 'bt_min_float'] = '50'
df_avg_per_month_bt_dup.loc[(df_avg_per_month_bt_dup['bt_min'] =='45'), 'bt_min_float'] = '75'
df_avg_per_month_bt_dup.loc[(df_avg_per_month_bt_dup['bt_min'] =='00'), 'bt_min_float'] = '00'

In [ ]:
df_avg_per_month_bt_dup['bt_float']=df_avg_per_month_bt_dup['bt_hr']+'.'+df_avg_per_month_bt_dup['bt_min_float']

In [ ]:
df_avg_per_month_wt_dup['wt_hr']=df_avg_per_month_wt_dup['Average_Waketime_Round_fifteenmin'].dt.strftime("%H").astype(str)
df_avg_per_month_wt_dup['wt_min']=df_avg_per_month_wt_dup['Average_Waketime_Round_fifteenmin'].dt.strftime("%M").astype(str)
df_avg_per_month_wt_dup.loc[(df_avg_per_month_wt_dup['wt_min'] =='15'), 'wt_min_float'] = '25'
df_avg_per_month_wt_dup.loc[(df_avg_per_month_wt_dup['wt_min'] =='30'), 'wt_min_float'] = '50'
df_avg_per_month_wt_dup.loc[(df_avg_per_month_wt_dup['wt_min'] =='45'), 'wt_min_float'] = '75'
df_avg_per_month_wt_dup.loc[(df_avg_per_month_wt_dup['wt_min'] =='00'), 'wt_min_float'] = '00'

In [ ]:
df_avg_per_month_wt_dup['wt_float']=df_avg_per_month_wt_dup['wt_hr']+'.'+df_avg_per_month_wt_dup['wt_min_float']

In [ ]:
#Wake figure
df_avg_per_month_wt_dup_order=df_avg_per_month_wt_dup
df_avg_per_month_wt_dup_order['wt_float']=df_avg_per_month_wt_dup_order['wt_float'].astype(float)


df_avg_per_month_wt_dup_order['flip_wake']=0

df_avg_per_month_wt_dup_order.loc[(df_avg_per_month_wt_dup_order['wt_float'] >12), 'flip_wake'] = df_avg_per_month_wt_dup_order['wt_float']-12

df_avg_per_month_wt_dup_order.loc[(df_avg_per_month_wt_dup_order['wt_float'] <12), 'flip_wake'] = df_avg_per_month_wt_dup_order['wt_float']+12
df_avg_per_month_wt_dup_order[['flip_wake','wt_float']]


In [ ]:
df_avg_per_month_wt_dup_order_fig2 = df_avg_per_month_wt_dup_order.drop_duplicates(subset=['User UUID', 'Average_Waketime_Round_fifteenmin','Month_Year_Name_Bedtime'], keep='first')


In [ ]:
fig_2_sort_wake=df_avg_per_month_wt_dup_order_fig2
fig_2_sort_wake=fig_2_sort_wake[(fig_2_sort_wake['closure_status'] =='During Closure')|(fig_2_sort_wake['closure_status'] =='Pre Closure')]

fig_2_sort_wake['Bedtime_Date_Month_Closure']=fig_2_sort_wake['Month_Year_Name_Bedtime'].str[0:3]
fig_2_sort_wake.loc[(fig_2_sort_wake['Month_Year_Name_Bedtime']=='1-14 Mar-2020'), 'Bedtime_Date_Month_Closure']='Mar'
fig_2_sort_wake.loc[(fig_2_sort_wake['Month_Year_Name_Bedtime']=='15-31 Mar-2020'), 'Bedtime_Date_Month_Closure']='Mar'
fig_2_sort_wake['month_only'] = pd.Categorical(fig_2_sort_wake['Bedtime_Date_Month_Closure'],
                                   categories=['Jan', 'Feb','Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec'],
                                   ordered=True)



In [ ]:

plt.figure(figsize=(95,35))
sns.set(font_scale =8)
sns.set_style("white")
#sns.set_palette("rocket_r", 8, .75)
#sns.color_palette("rocket_r")

palette = {'Pre Closure':"indianred",
            #2020:"paleturquoise"
           'During Closure':"gold"}
ax=sns.boxplot(x = 'month_only', y = 'flip_wake',
               data = fig_2_sort_wake
           ,hue='closure_status'
           , palette=palette
              , linewidth=12,
                showfliers=False
                      )
plt.setp(ax.get_xticklabels(), rotation=0)
plt.yticks([17.,17.5,18.,18.5,19.,19.5,20,20.5,21.,21.5,22.,22.5],['5 AM','5:30 AM','6 AM','6:30 AM','7 AM','7:30 AM','8 AM','8:30 AM','9 AM','9:30 AM','10 AM','10:30 AM'])
ax.set_xlabel(" ",fontdict= { 'fontsize': 90, 'fontweight':'bold'}, y =4.5)
ax.set_ylabel("Average Daily Wake Time",fontdict= { 'fontsize': 100, 'fontweight':'bold'})
ax.set_title('Average Daily Wake Times By Month (rounded to nearest 15 mins) for Participants(Night Sleep)',fontdict= { 'fontsize': 90, 'fontweight':'bold'}, y =1.03)
plt.legend(bbox_to_anchor=(1.01,0.5), loc="center left", borderaxespad=0,frameon=True,edgecolor='black'
           ,title='Year',framealpha=0.5)
plt.setp(ax.get_legend().get_texts(), fontsize='80') # for legend text
plt.setp(ax.get_legend().get_title(), fontsize='80') # for legend title


In [ ]:

df_avg_per_month_bt_dup_order=df_avg_per_month_bt_dup


In [ ]:

df_avg_per_month_bt_dup_order=df_avg_per_month_bt_dup
df_avg_per_month_bt_dup_order['bt_float']=df_avg_per_month_bt_dup_order['bt_float'].astype(float)

df_avg_per_month_bt_dup_order['flip']=0

df_avg_per_month_bt_dup_order.loc[(df_avg_per_month_bt_dup_order['bt_float'] >12), 'flip'] = df_avg_per_month_bt_dup_order['bt_float']-12

df_avg_per_month_bt_dup_order.loc[(df_avg_per_month_bt_dup_order['bt_float'] <12), 'flip'] = df_avg_per_month_bt_dup_order['bt_float']+12

In [ ]:

#Based on Bedtime_Date_Month

df_avg_per_month_bt_dup_order_fig2 = df_avg_per_month_bt_dup_order.drop_duplicates(subset=['User UUID', 'Average_Bedtime_Round_fifteenmin','Month_Year_Name_Bedtime'], keep='first')


In [ ]:

fig_2_sort=df_avg_per_month_bt_dup_order_fig2
fig_2_sort=fig_2_sort[(fig_2_sort['closure_status'] =='During Closure')|(fig_2_sort['closure_status'] =='Pre Closure')]

fig_2_sort['Bedtime_Date_Month_Closure']=fig_2_sort['Month_Year_Name_Bedtime'].str[0:3]
fig_2_sort.loc[(fig_2_sort['Month_Year_Name_Bedtime']=='1-14 Mar-2020'), 'Bedtime_Date_Month_Closure']='Mar'
fig_2_sort.loc[(fig_2_sort['Month_Year_Name_Bedtime']=='15-31 Mar-2020'), 'Bedtime_Date_Month_Closure']='Mar'
fig_2_sort['month_only'] = pd.Categorical(fig_2_sort['Bedtime_Date_Month_Closure'],
                                   categories=['Jan', 'Feb','Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec'],
                                   ordered=True)


fig_2_sort['Month_Only_abb']=fig_2_sort['Start_Time_Offset'].dt.strftime('%b')

fig_2_sort['Bedtime_Date_Month']=pd.to_datetime(fig_2_sort['Bedtime_Date']).dt.strftime('%b')


In [ ]:
#based on Bedtime_Date_Month final

plt.figure(figsize=(95,35))
sns.set(font_scale =8)
sns.set_style("white")

palette = {'Pre Closure':"steelblue",
           'During Closure':"skyblue"
           }
ax=sns.boxplot(x = 'month_only', y = 'flip'
               , data = fig_2_sort
           ,hue='closure_status'
           , palette=palette
              , linewidth=12,
                showfliers=False
            )
plt.setp(ax.get_xticklabels(), rotation=0)
plt.yticks([8.,9.,10.,11.,12.,13.,14.,15.,16.],['8 PM','9 PM','10 PM','11 PM','12 AM','1 AM','2 AM','3 AM','4 AM'])
ax.set_xlabel(" ",fontdict= { 'fontsize': 90, 'fontweight':'bold'}, y =4.5)
ax.set_ylabel("Average Daily Bed Time",fontdict= { 'fontsize': 100, 'fontweight':'bold'})
ax.set_title('Average Daily Sleep Times By Month (rounded to nearest 15 mins) for Participants(Night Sleep)',fontdict= { 'fontsize': 90, 'fontweight':'bold'}, y =1.03)
plt.legend(bbox_to_anchor=(1.01,0.5), loc="center left", borderaxespad=0,frameon=True,edgecolor='black'
           ,title='Year',framealpha=0.5)
plt.setp(ax.get_legend().get_texts(), fontsize='80') # for legend text
plt.setp(ax.get_legend().get_title(), fontsize='80') # for legend title


In [ ]:



df_sleep_bedtime_calc_allnight_val_mean_order['flip_wake']=0

df_sleep_bedtime_calc_allnight_val_mean_order.loc[(df_sleep_bedtime_calc_allnight_val_mean_order['Average_Waketime_Round_thirtymin_Float'] >12), 'flip_wake'] = df_sleep_bedtime_calc_allnight_val_mean_order['Average_Waketime_Round_thirtymin_Float']-12

df_sleep_bedtime_calc_allnight_val_mean_order.loc[(df_sleep_bedtime_calc_allnight_val_mean_order['Average_Waketime_Round_thirtymin_Float'] <12), 'flip_wake'] = df_sleep_bedtime_calc_allnight_val_mean_order['Average_Waketime_Round_thirtymin_Float']+12

df_sleep_bedtime_calc_allnight_val_mean_order[['flip','Average_Bedtime_Round_thirtymin_Float']]

df_sleep_bedtime_calc_allnight_val_mean_order[['flip_wake','Average_Waketime_Round_thirtymin_Float']]


In [ ]:
df_sleep_bedtime_calc_allnight_val_mean_dup = df_sleep_bedtime_calc_allnight_val_mean.drop_duplicates(subset=['User UUID', 'Average_Bedtime_Round_thirtymin_Float','Month_Year_Name_Bedtime'], keep='first')


df_sleep_waketime_calc_allnight_val_mean_dup = df_sleep_bedtime_calc_allnight_val_mean.drop_duplicates(subset=['User UUID', 'Average_Waketime_Round_thirtymin','Month_Year_Name_Bedtime'], keep='first')


In [ ]:
#Sleep duration

In [ ]:


df_sleep_bedtime_calc_allnight_val_mean_order=df_sleep_bedtime_calc_allnight_val_mean

#Sleep duration

df_sleep_duration=df_sleep_bedtime_calc_allnight_val_mean_order


df_sleep_duration['Total_Sleep_Per_Day_Bedtime'] = df_sleep_duration.groupby(['Bedtime_Date', 'User UUID'])['Value'].transform('sum')

df_sleep_duration['Total_Sleep_Per_Day_Bedtime_float']=df_sleep_duration['Total_Sleep_Per_Day_Bedtime'].astype(float)

df_sleep_duration['Total_Sleep_Per_Day_Bedtime_float_Hours']=df_sleep_duration['Total_Sleep_Per_Day_Bedtime_float']/3600

df_sleep_duration['Average_Monthly_Sleep_Duration_Bedtime'] = df_sleep_duration.groupby([ 'User UUID','Month_Year_Name_Bedtime'])['Total_Sleep_Per_Day_Bedtime_float_Hours'].transform('mean')



df_sleep_duration_fig1 = df_sleep_duration.drop_duplicates(subset=['User UUID', 'Average_Monthly_Sleep_Duration_Bedtime','Month_Year_Name_Bedtime'], keep='first')


In [ ]:


#Min
df_sleep_duration_fig1.groupby('closure_status')['Average_Monthly_Sleep_Duration_Bedtime'].min()




In [ ]:

#Max
df_sleep_duration_fig1.groupby('closure_status')['Average_Monthly_Sleep_Duration_Bedtime'].max()



In [ ]:

#Mean
df_sleep_duration_fig1.groupby('closure_status')['Average_Monthly_Sleep_Duration_Bedtime'].mean()



In [ ]:

#Std
df_sleep_duration_fig1.groupby('closure_status')['Average_Monthly_Sleep_Duration_Bedtime'].std()



In [ ]:


#Users
df_sleep_duration_fig1.groupby('closure_status')['User UUID'].nunique()

In [ ]:
df_sleep_duration_month=df_sleep_duration

df_sleep_duration_month['Avg_Sleep_Duration_Per_Month'] = df_sleep_duration_month.groupby(['Month_Year_Name_Bedtime'])['Average_Monthly_Sleep_Duration_Bedtime'].transform('mean')


df_sleep_duration_month = df_sleep_duration_month.drop_duplicates(subset=['Avg_Sleep_Duration_Per_Month','Month_Year_Name_Bedtime'], keep='first')


In [ ]:
#Min
df_sleep_duration_month.groupby('closure_status')['Avg_Sleep_Duration_Per_Month'].min()

In [ ]:

#Max
df_sleep_duration_month.groupby('closure_status')['Avg_Sleep_Duration_Per_Month'].max()

In [ ]:

#Mean
df_sleep_duration_month.groupby('closure_status')['Avg_Sleep_Duration_Per_Month'].mean()


In [ ]:
#Std
df_sleep_duration_month.groupby('closure_status')['Avg_Sleep_Duration_Per_Month'].std()

In [ ]:
#Statistical test

sleep_pre_closure=df_sleep_duration_month[df_sleep_duration_month['closure_status']=='Pre Closure']

sleep_pre_closure['Bedtime_Date']=pd.to_datetime(sleep_pre_closure['Bedtime_Date'])

sleep_pre_closure['month_num']=sleep_pre_closure['Bedtime_Date'].dt.month

sleep_pre_closure=sleep_pre_closure.sort_values(by=['month_num'])

sleep_pre_closure_dup = sleep_pre_closure.drop_duplicates(subset=['Avg_Sleep_Duration_Per_Month', 'Month_Year_Name_Bedtime'], keep='first')
sleep_pre_closure_dup=sleep_pre_closure_dup[['Avg_Sleep_Duration_Per_Month', 'Month_Year_Name_Bedtime']]



In [ ]:
#Statistical test

sleep_dur_closure=df_sleep_duration_month[df_sleep_duration_month['closure_status']=='During Closure']

sleep_dur_closure['Bedtime_Date']=pd.to_datetime(sleep_dur_closure['Bedtime_Date'])

sleep_dur_closure['month_num']=sleep_dur_closure['Bedtime_Date'].dt.month

sleep_dur_closure=sleep_dur_closure.sort_values(by=['month_num'])


sleep_dur_closure_dup = sleep_dur_closure.drop_duplicates(subset=['Avg_Sleep_Duration_Per_Month', 'Month_Year_Name_Bedtime'], keep='first')
sleep_dur_closure_dup=sleep_dur_closure_dup[['Avg_Sleep_Duration_Per_Month', 'Month_Year_Name_Bedtime']]


In [ ]:

from scipy.stats import mannwhitneyu
U1, p = mannwhitneyu(sleep_dur_closure_dup['Avg_Sleep_Duration_Per_Month'], sleep_pre_closure_dup['Avg_Sleep_Duration_Per_Month'])
print(U1)
#print(p*100)

print(p)

In [ ]:
#Figure

In [ ]:
fig_duration_sort=df_sleep_duration_fig1

fig_duration_sort=fig_duration_sort[(fig_duration_sort['closure_status'] =='During Closure')|(fig_duration_sort['closure_status'] =='Pre Closure')]

fig_duration_sort['Bedtime_Date_Month_Closure']=fig_duration_sort['Month_Year_Name_Bedtime'].str[0:3]
fig_duration_sort.loc[(fig_duration_sort['Month_Year_Name_Bedtime']=='1-14 Mar-2020'), 'Bedtime_Date_Month_Closure']='Mar'
fig_duration_sort.loc[(fig_duration_sort['Month_Year_Name_Bedtime']=='15-31 Mar-2020'), 'Bedtime_Date_Month_Closure']='Mar'

fig_duration_sort['month_only'] = pd.Categorical(fig_duration_sort['Bedtime_Date_Month_Closure'],
                                   categories=['Jan', 'Feb','Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec'],
                                   ordered=True)

In [ ]:

plt.figure(figsize=(100,40))
sns.set(font_scale =8)
sns.set_style("white")
#sns.set_palette("rocket_r", 8, .75)
#sns.color_palette("rocket_r")
palette = {'Pre Closure':"yellowgreen",
           'During Closure':"pink"
           }
ax=sns.boxplot(x = 'month_only', y = 'Average_Monthly_Sleep_Duration_Bedtime'
               , data = fig_duration_sort
           ,hue='closure_status'
           , palette=palette
              , linewidth=14,
                showfliers=False
                       )
plt.setp(ax.get_xticklabels(), rotation=0)
ax.set_xlabel(" ",fontdict= { 'fontsize': 90, 'fontweight':'bold'}, y =4.5)
ax.set_ylabel("Average Daily Sleep Duration (Hours)",fontdict= { 'fontsize': 100, 'fontweight':'bold'})
ax.set_title('Average Daily Sleep Duration By Month (rounded to nearest 30 mins) for Participants(Night Sleep)',fontdict= { 'fontsize': 90, 'fontweight':'bold'}, y =1.03)
plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0,frameon=True,edgecolor='black'
           ,title='Year',framealpha=0.5)
plt.setp(ax.get_legend().get_texts(), fontsize='90') # for legend text
plt.setp(ax.get_legend().get_title(), fontsize='90') # for legend title
